In [5]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import re
import plotly
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,scorer
from sklearn.metrics import f1_score
import statsmodels.api as sm
from sklearn.metrics import precision_score,recall_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
import pickle as pkl

# How to manage One Hot VS Mean Encoding double modeling
see by hand which model between the couple is the best and run it, otherwise the work will be doubled and it is not interesting to present a worst perfoming for the ones that are reading.
I have already done for you, maybe double check it, but the pseudo table that I created is reliable 

# Function to create metrics matrix (also called MAETRIX)

In [2]:
def rapporto_modello(modello,train_x,test_x,train_y,test_y,nome) :
    modello.fit(train_x,train_y)
    predictions  = model.predict(test_x)
    #qui metriche
    Accuracy     = accuracy_score(test_y,predictions)
    Recall  = recall_score(test_y,predictions)
    precision    = precision_score(test_y,predictions)
    roc_auc      = roc_auc_score(test_y,predictions)
    F1_score      = f1_score(test_y,predictions) 
    
    df = pd.DataFrame({"Models"           : [name],
                       "Accuracy"  : [accuracy],
                       "Recall"    : [recallscore],
                       "Precision"       : [precision],
                       "F1_score"        : [f1score],
                       "AUC_score": [roc_auc]
                      })
    return df
# bisogna tu dia un mome diverso ad ogni modello anche perchè vengono sovrascritti uno sull'altro
# i modelli vengono generati non all'interno di una pipeline, 
# altrimenti sarebbe stato più facile e avrei preso come input solo il parametro modello



model_SVM = rapporto_modello(model_svm,train_x,test_x,train_y, test_y,"SVM")

model_LOG= rapporto_modello(model_log, train_x, test_x, train_y, test_y,"Logistics")

model_GDB = rapporto_modello(model_gdb, train_x, test_x, train_y, test_y,"Gradient_Boosting")

model_RF = rapporto_modello(model_rf,rain_x, test_x, train_y, test_y,"Random_Forest")

model_XGB = rapporto_modello(model_xgb,rain_x, test_x, train_y, test_y,"XGBOOST")
    
model_performances = pd.concat([model_SVM,model_LOG,model_GDB,
                                model_RF,model_XGB],axis = 0).reset_index()


SyntaxError: positional argument follows keyword argument (<ipython-input-2-d59f23aac30a>, line 28)

In [ ]:
# mai testata non so se funziona
raccolta_modelli = [model_svm, model_log, model_gdb, model_rf, model_xgb]
def dammi_tabella(train_x,test_x,train_y,test_y,nome):
    best={}
    for i in raccolta_modelli:
        best["".format(i)] = rapporto_modello(i,train_x,test_x,train_y, test_y,"i")


        model_performances = pd.concat([model_SVM,model_LOG,model_GDB,
                                    model_RF,model_XGB],axis = 0).reset_index()
    return model_performances

# Create pseudo-dataframe

In [3]:
# i dati non sono casuali,sono presi dal confronto tra One Hot 
# e Mean Ending di ogni modello sulla base dello f1 maggiore  
# XGBOOST non ancora tunato su treshold


data = {'Models' : ['Logistic Regression','Gradient Boosting', 'Support vector classifier', 'Random Forest', 'XGBOOST'],
         'AUC_score':  [0.8264, 0.8314, 0.7700, 0.8293, 0.8337],
         'Precision': [0.5092, 0.5265, 0.4449, 0.5355, 0.5],
         'Recall': [0.8101, 0.7433, 0.7566, 0.7647, 0.8288],
         'Accuracy': [0.7420, 0.7540, 0.6844, 0.7611, 0.7341],
         'F1_Score': [0.6253, 0.6164, 0.5603, 0.6299, 0.6237]

    }
df = pd.DataFrame(data, columns = ['AUC_score','Precision', 'Recall', 'Accuracy', 'F1_Score','Models'])
df


,AUC_score,Precision,Recall,Accuracy,F1_Score,Models
0,0.8264,0.5092,0.8101,0.7420,0.6253,Logistic Regression
1,0.8314,0.5265,0.7433,0.7540,0.6164,Gradient Boosting
2,0.7700,0.4449,0.7566,0.6844,0.5603,Support vector classifier
3,0.8293,0.5355,0.7647,0.7611,0.6299,Random Forest
4,0.8337,0.5000,0.8288,0.7341,0.6237,XGBOOST


# Def Plot Radar 

In [13]:
import plotly.graph_objects as go

# categories =['AUC score', 'Precision', 'Recall', 'Accuracy', 'F1 Score']


######################################

def plot_radar(df,title):
    fig = go.Figure()
    categories =df.drop('Models',axis = 1).columns
    
    fig.add_trace(go.Scatterpolar(
          r=df.loc[0,['AUC_score','Precision','Recall','Accuracy','F1_Score']],
          theta=categories,
          fill='toself',
          name = df.Models[0],
        mode = "markers+lines"
    )),
    fig.add_trace(go.Scatterpolar(
          r=df.loc[1,['AUC_score','Precision','Recall','Accuracy','F1_Score']],
          theta=categories,
          fill='toself',
          name=df.Models[1]
    )),

    fig.add_trace(go.Scatterpolar(
          r=df.loc[2,['AUC_score','Precision','Recall','Accuracy','F1_Score']],
          theta=categories,
          fill='toself',
          name=df.Models[2]
    )),
    fig.add_trace(go.Scatterpolar(
          r=df.loc[3,['AUC_score','Precision','Recall','Accuracy','F1_Score']],
          theta=categories,
          fill='toself',
          name=df.Models[3]
    )),

    fig.add_trace(go.Scatterpolar(
          r=df.loc[4,['AUC_score','Precision','Recall','Accuracy','F1_Score']],
          theta=categories,
          fill='toself',
          name=df.Models[4]
    ))

    #############################
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                showline = True,
                linewidth = 2,
                tickwidth = 4,
                gridcolor = "white",
                gridwidth = 2,
                range=[0.3, 0.9]),
            angularaxis = dict(
                tickfont = dict(size = 13),
                layer = "above traces"),
            bgcolor  = "rgb(243,243,243)",  
        ),
        paper_bgcolor = "rgb(243,243,243)",
        title = title,
        height = 700,
        showlegend=True
    )
    fig.show()
    
plot_radar(df,'RADAR Plot')